In [ ]:
# Imports: DO NOT CHANGE
import cv2
import numpy as np
from numpy.typing import NDArray
from scipy.signal import convolve2d
from typing import Tuple, List
from scipy import datasets
from matplotlib import pyplot as plt

In [ ]:
# Axiliary functions. DO NOT CHANGE
def generate_chessboard_pattern(rows: int, cols: int, square_size: int = 100):
    # Create an empty white image with the desired dimensions
    image = np.ones((rows * square_size, cols * square_size, 3), dtype=np.uint8) * 255

    # Loop through each square of the chessboard and assign colors alternately
    for i in range(rows):
        for j in range(cols):
            if (i + j) % 2 == 0:
                color = (0, 0, 0)  # Black color for even squares
            else:
                color = (255, 255, 255)  # White color for odd squares
            cv2.rectangle(image, (j * square_size, i * square_size), ((j + 1) * square_size, (i + 1) * square_size), color, -1)
    return image

def normalize_image(image: NDArray, min:int = 0, max:int = 255):
    img = (image - image.min()) * (max - min) / (image.max() - image.min()) + min
    return img


def plot_image_row(images: List[NDArray], fig_width = 5):
    
    number_of_images = len(images)
    fig_size = (fig_width, fig_width/3)
    if number_of_images > 0:
        image_y_size = images[0].shape[0]
        image_x_size = images[0].shape[1]
        fig_size = (fig_width * number_of_images, fig_width * image_y_size / image_x_size)

    fig, ax = plt.subplots(1, number_of_images, figsize=fig_size, squeeze=False)

    for image_index in range(number_of_images):
        ax[0, image_index].imshow(images[image_index], cmap="gist_gray")
    
    return fig, ax

In [ ]:
def compute_derivatives(image: NDArray) -> Tuple[NDArray, NDArray]:
    # TODO: Reemplazar con 3x3 Sobel filter en dirección "y"
    sobel_y = convolve2d(np.array([[1, 0, -1]]).T, np.array([[1, 2, 1]]))
    # sobel_y = np.eye(3)
    # Como sobel_x y sobel_y son simétricos, el uno es la transpuesta del otro
    sobel_x = sobel_y.T
    
    # TODO: Reemplazar las dos lineas con laderivada utilizando el operador de convolución
    Ix = convolve2d(image, sobel_x, mode="same", boundary="symm")
    Iy = convolve2d(image, sobel_y, mode="same", boundary="symm")

    return Ix, Iy

In [ ]:
# Visualicemos derivadas
chessboard_image = generate_chessboard_pattern(6, 8)
chessboard_image = cv2.cvtColor(chessboard_image, cv2.COLOR_RGB2GRAY)
racoon_image = datasets.face().astype('uint8')
racoon_image = cv2.cvtColor(racoon_image, cv2.COLOR_RGB2GRAY)

print(f"Chess shape{chessboard_image.shape}")
chess_Ix, chess_Iy = compute_derivatives(chessboard_image)
racoon_Ix, racoon_Iy = compute_derivatives(racoon_image)

plot_image_row([chessboard_image, chess_Ix, chess_Iy])
plot_image_row([racoon_image, racoon_Ix, racoon_Iy])

In [ ]:
def compute_eigen_vector_maps(Ix: NDArray, Iy:  NDArray, blockSize=3) -> Tuple[NDArray]:
    """_summary_

    Args:
        Ix (NDArray): Mapa de derivada en dirección x
        Iy (NDArray): Mapa de derivada en dirección y
        IxIy (NDArray): Mapa de multiplicación de derivadas en x y y
        blockSize (int, optional): Tamaño de bloque para respuesta de harris. Defaults to 3.

    Returns:
        Tuple[NDArray]: Máximo y mínimo de valores propios
    """
    lambda_min = np.zeros_like(Ix)
    lambda_max = np.zeros_like(Ix)

    Ix2 = Ix * Ix
    Iy2 = Iy * Iy
    IxIy = Ix * Iy

    # TODO: Reemplazar las siguientes tres líneas para computar mapas que computen suma de sobre el kernel de tamaño "blockSize" para cada pixel
    summed_Ix2 = convolve2d(Ix2, np.ones((blockSize, blockSize)), mode="same", boundary="wrap")
    summed_Iy2 = convolve2d(Iy2, np.ones((blockSize, blockSize)), mode="same", boundary="wrap")
    summed_IxIy = convolve2d(IxIy, np.ones((blockSize, blockSize)), mode="same", boundary="wrap")

    for index_y in range(Ix.shape[0]):
        for index_x in range(Ix.shape[1]):
            mat = np.array([[summed_Ix2[index_y, index_x], summed_IxIy[index_y, index_x]],
                            [summed_IxIy[index_y, index_x], summed_Iy2[index_y, index_x]]])
            

            # TODO: computar valores propios de la matriz mat y asignar sus valores a las matrices lambda_min, lambda_max
            e_vals, _ = np.linalg.eig(mat)
            e_vals = sorted(e_vals)
            lambda_min[index_y, index_x] = e_vals[0]
            lambda_max[index_y, index_x] = e_vals[1]

    return lambda_min, lambda_max

In [ ]:
# Visualizemos los mapas de valores propios
chess_lambda_min, chess_lambda_max = compute_eigen_vector_maps(chess_Ix, chess_Iy)
racoon_lambda_min, racoon_lambda_max = compute_eigen_vector_maps(racoon_Ix, racoon_Iy)

plot_image_row([chessboard_image, chess_lambda_max, chess_lambda_min])
plot_image_row([racoon_image, racoon_lambda_max, racoon_lambda_min])


In [ ]:

def compute_harris(Ix: NDArray, Iy: NDArray, k=0.06, blockSize=3) -> NDArray:
    """ Computes harris response

    Args:
        Ix (NDArray): Mapa de derivada en dirección x.
        Iy (NDArray): Mapa de derivada en dirección y.
        IxIy (NDArray): Mapa de multiplicación de derivadas en x y y.
        k (float, optional): Parámetro para respuesta de harris. Defaults to 0.06.
        blockSize (int, optional): Tamaño de bloque para respuesta de harris. Defaults to 3.

    Returns:
        NDArray: Imagen de respuesta de harris
    """
    Ix2 = Ix * Ix
    Iy2 = Iy * Iy
    IxIy = Ix * Iy
    summed_Ix2 = convolve2d(Ix2, np.ones((blockSize, blockSize)), mode="same", boundary="wrap")
    summed_Iy2 = convolve2d(Iy2, np.ones((blockSize, blockSize)), mode="same", boundary="wrap")
    summed_IxIy = convolve2d(IxIy, np.ones((blockSize, blockSize)), mode="same", boundary="wrap")
    
    # TODO: Computar respuesta de harris dados Ix2, Iy2 y IxIy
    trace = summed_Ix2 + summed_Iy2
    resp = summed_Ix2 * summed_Iy2 - summed_IxIy*summed_IxIy - k * trace * trace
    return resp

In [ ]:
# Visualizamos la respuesta de Harris
chess_harrris = compute_harris(chess_Ix, chess_Iy)
racoon_harris = compute_harris(racoon_Ix, racoon_Iy)

plot_image_row([chessboard_image, chess_harrris])
plot_image_row([racoon_image, racoon_harris], fig_width=10)